### Word In Visual Genome Process

In [ ]:
#!pip install --upgrade pip

In [ ]:
#!pip install --upgrade regex

In [ ]:
#!pip install --upgrade pandas

In [ ]:
#!pip install --upgrade yellowbrick

In [ ]:
#!pip install --upgrade scipy

In [ ]:
#import re
#print(re.__version__)

In [ ]:
#import pandas
#print(pandas.__version__)

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [ ]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/4-Word In Visual Genome"

Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [ ]:
def word_group_dataframe2(df, search_list, target_column):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)]
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

#### Visual Genome Data

In [ ]:
df_genome_objects = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Objects_Analysis.csv")
df_genome_objects

In [ ]:
df_genome_objects["names"].nunique()

In [ ]:
df_genome_attributes = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Attributes_Analysis.csv")
df_genome_attributes

In [ ]:
df_genome_relationships = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Relationships_Analysis.csv")
df_genome_relationships

#### Word Search In Genome Data

In [ ]:
df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Result/2-Word Select Process/{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
df_word_select

In [ ]:
word_list = ['an','anlama','doğum','gidiyor','günün','istediğim','istediğin','istediğiniz','kutlu','olmuş','soru','söylemek','zamanki','zor','şuna']

In [ ]:
df_word_raw = df_word_select[df_word_select["word"].isin(word_list)]
df_word_raw.reset_index(drop=True, inplace=True)
df_word_raw.drop(["frequency","stem"], inplace=True, axis=1)
df_word_raw

In [ ]:
df_word_raw = pd.read_excel("Turkish_45000_Process.xlsx")
df_word_raw

In [ ]:
df_word_raw_lemma_trans_list = list(set(df_word_raw["lemma_en_translate"]))
df_word_raw_word_trans_list = list(set(df_word_raw["word_en_translate"]))

In [ ]:
len(df_word_raw_lemma_trans_list)

In [ ]:
len(df_word_raw_word_trans_list)

In [ ]:
#df_object_search_word_result = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 20)
df_object_search_word_result = word_group_dataframe2(df_genome_objects, df_word_raw_word_trans_list, "names")
df_object_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_object_search_word_result

In [ ]:
df_object_search_word_result["word_en_translate"].nunique()

In [ ]:
df_object_search_word_result_merge = pd.merge(df_object_search_word_result,df_word_raw,how="left",on="word_en_translate")
df_object_search_word_result_merge

In [ ]:
df_word_count = df_object_search_word_result_merge.groupby(["word_en_translate"])[["names"]].count().reset_index()
df_word_count.rename(columns={"names":"names_count"}, inplace=True)
df_word_count.sort_values(by="names_count",ascending=False, inplace=True)
df_word_count.reset_index(drop=True, inplace=True)
df_word_count

In [ ]:
df_object_search_word_result_merge.to_excel("1000_Words_Word_Object_Names.xlsx", index=False)

In [ ]:
#df_object_search_lemma_result = word_group_dataframe(df_genome_objects, df_word_raw_lemma_trans_list, "names", 10)
df_object_search_lemma_result = word_group_dataframe2(df_genome_objects, df_word_raw_lemma_trans_list, "names")
df_object_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_object_search_lemma_result

In [ ]:
df_object_search_lemma_result["lemma_en_translate"].nunique()

In [ ]:
df_object_search_lemma_result_merge = pd.merge(df_object_search_lemma_result,df_word_raw,how="left",on="lemma_en_translate")
df_object_search_lemma_result_merge

In [ ]:
df_lemma_count = df_object_search_lemma_result_merge.groupby(["lemma_en_translate"])[["image_id"]].count().reset_index()
df_lemma_count.rename(columns={"image_id":"image_id_count"}, inplace=True)
df_lemma_count.sort_values(by="image_id_count",ascending=False, inplace=True)
df_lemma_count.reset_index(drop=True, inplace=True)
df_lemma_count

In [ ]:
df_object_search_lemma_result_merge.to_excel("1000_Words_Lemma_Object_Names.xlsx", index=False)

In [ ]:
df_lemma_count.to_excel("1000_Words_Lemma_Object_Names_Count.xlsx", index=False)

In [ ]:

lemma_count_list = list(set(df_lemma_count["lemma_en_translate"]))

In [ ]:
len(lemma_count_list)

In [ ]:
df_word_raw

In [ ]:
df_lemma_not_cover = df_word_raw[~df_word_raw["lemma_en_translate"].isin(lemma_count_list)]
df_lemma_not_cover

In [ ]:
lemma_not_cover_set = set(df_lemma_not_cover["word"])
lemma_not_cover_set

In [ ]:
#df_object_search_word_result = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 20)
df_object_search_word_result = word_group_dataframe2(df_genome_objects, df_word_raw_word_trans_list, "names")
df_object_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_object_search_word_result

In [ ]:
df_object_search_word_result["word_en_translate"].nunique()

In [ ]:
df_object_search_word_result_merge = pd.merge(df_object_search_word_result,df_word_raw,how="left",on="word_en_translate")
df_object_search_word_result_merge

In [ ]:
df_word_count = df_object_search_word_result_merge.groupby(["word_en_translate"])[["names"]].count().reset_index()
df_word_count.rename(columns={"names":"names_count"}, inplace=True)
df_word_count.sort_values(by="names_count",ascending=False, inplace=True)
df_word_count.reset_index(drop=True, inplace=True)
df_word_count

In [ ]:
df_object_search_word_result_merge.to_excel("1000_Words_Word_Object_Names.xlsx", index=False)

In [ ]:
df_word_count.to_excel("1000_Words_Word_Object_Names_Count.xlsx", index=False)

In [ ]:
word_count_set = list(set(df_word_count["word_en_translate"]))

In [ ]:
len(word_count_set)

In [ ]:
df_word_cover = df_word_raw[df_word_raw["word_en_translate"].isin(word_count_set)]
df_word_cover

In [ ]:
word_cover_set = set(df_word_cover["word"])
word_cover_set

In [ ]:
word_cover_set.intersection(lemma_not_cover_set)

In [ ]:
len(word_cover_set.intersection(lemma_not_cover_set))

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
output_file

In [ ]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass